# Checkpoint 3

----------------------
In this notebook, I continue the topic modeling I did for my last progress report. I attempt to use several different models for topic modeling, including bigram and trigram models (note that those models aren't purely bigram/trigrams, but only include bigrams and trigrams that occur multiple times). I also look at the coherence and perplexity scores of the models to evaluate their performances.

In [2]:
cd C:\Users\Patrick\Dropbox\Documents\Sophomore Year\Semester_2\Ling_1340_Data_Science_for_Linguists\Project\gensim

C:\Users\Patrick\Dropbox\Documents\Sophomore Year\Semester_2\Ling_1340_Data_Science_for_Linguists\Project\gensim


### Trying different models

In this section I compare scores for unigram, bigram and trigram models. Bigrams and trigrams are only replaced if the bigram/trigram is frequent enough. Also trigrams technically includes 4-grams because it includes any pairs from the bigram model, which can be two bigrams (i.e. 4 words) or a bigarm and a unigram (i.e. 3 words).

In [3]:
#first, get my pickle file back (i call it df but i'm p sure its a series)
import pickle

with open('all_lemmatized_series.pkl','rb') as f:
    df = pickle.load(f)
    
#make a dictionary, use the dictionary to make a bag of words corpus
from gensim.corpora import Dictionary
dictionary = Dictionary(df) #finds allunique words i think
bow_corp = [dictionary.doc2bow(file) for file in df] 

In [5]:
import gensim

##these section uses gensim.models.phrases to make bigrams and trigrams basically the first step
##joins words that occur together multiple times... the next makes faster models for the 
##bigrams/trigrams, and the final step inserts them 

bigram = gensim.models.Phrases(df, threshold = 5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[df], threshold = 5)  

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

df_bigrams = [bigram_mod[doc] for doc in df]
df_trigrams = [trigram_mod[bigram_mod[doc]] for doc in df]

In [6]:
##make the lda_model. This is an "online" lda model bc it updates every 100 document and
##only passes a few times through, last time I used a batch model which updates after all
##the documents and runs more times (which is slower and not significantly better afaik)
lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corp,
                                            id2word=dictionary,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           random_state= 100,
                                           alpha='auto',    
                                           per_word_topics=True)

#Perplexity is a measure of how good the model is. The lower the better.
print('\nPerplexity: ', lda_model.log_perplexity(bow_corp))  

# Compute Coherence Score. there are a bunch of types but i used u_mass which is supposedly one of the faster models
# iirc, lower u_mass coherence is better, but it may also be the opposite I have to check
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=df, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -12.892946261137208

Coherence Score:  -4.6520907968147736


In [7]:
##visualize the model with pyLDAvis
import pyLDAvis, pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corp, dictionary)
vis

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nose\config.py:178: DeprecationWarning: invalid escape sequence \.
  """
C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.312016  0.024640       1        1  39.470928
3      0.222170 -0.059432       2        1  15.448947
16     0.233092  0.101456       3        1   7.755843
5      0.237676  0.225970       4        1   7.335402
10     0.217713  0.004546       5        1   7.191001
7      0.136232 -0.228976       6        1   6.895652
15     0.152314 -0.208719       7        1   3.066457
13     0.121954  0.200682       8        1   2.790557
4      0.065095 -0.166774       9        1   2.304275
14     0.028357  0.044002      10        1   1.748542
12    -0.049262 -0.075769      11        1   1.574022
1     -0.115457  0.032104      12        1   1.079164
8     -0.151211  0.009342      13        1   0.927529
17    -0.117743 -0.015770      14        1   0.867551
0     -0.170887  0.003647      15        1   0.511157
9     -0.203717  0.019990      16        1   0.453357
2     -0.214094  0.020337      17        1   0.394587
6     -0.236077  0.022590      18        1   0.080252
18    -0.233326  0.023234      19        1   0.053904
19    -0.234847  0.022898      20        1   0.050877, topic_info=      Category          Freq   ...     loglift  logprob
term                           ...                     
2916   Default   7294.000000   ...     30.0000  30.0000
57     Default  16003.000000   ...     29.0000  29.0000
138    Default   5609.000000   ...     28.0000  28.0000
130    Default   9304.000000   ...     27.0000  27.0000
10923  Default   2744.000000   ...     26.0000  26.0000
1020   Default   3395.000000   ...     25.0000  25.0000
836    Default   6113.000000   ...     24.0000  24.0000
1102   Default   3151.000000   ...     23.0000  23.0000
409    Default   4582.000000   ...     22.0000  22.0000
0      Default   6644.000000   ...     21.0000  21.0000
2124   Default   3884.000000   ...     20.0000  20.0000
300    Default   2541.000000   ...     19.0000  19.0000
990    Default   5713.000000   ...     18.0000  18.0000
2194   Default   2080.000000   ...     17.0000  17.0000
124    Default   3627.000000   ...     16.0000  16.0000
2631   Default   2346.000000   ...     15.0000  15.0000
391    Default   3499.000000   ...     14.0000  14.0000
11823  Default   2841.000000   ...     13.0000  13.0000
587    Default   5788.000000   ...     12.0000  12.0000
822    Default   1607.000000   ...     11.0000  11.0000
1658   Default   3318.000000   ...     10.0000  10.0000
1      Default   4442.000000   ...      9.0000   9.0000
2581   Default   3465.000000   ...      8.0000   8.0000
2331   Default   4313.000000   ...      7.0000   7.0000
1393   Default   2143.000000   ...      6.0000   6.0000
1306   Default   2896.000000   ...      5.0000   5.0000
1367   Default   2390.000000   ...      4.0000   4.0000
4675   Default   1556.000000   ...      3.0000   3.0000
11498  Default   1931.000000   ...      2.0000   2.0000
258    Default   4858.000000   ...      1.0000   1.0000
...        ...           ...   ...         ...      ...
23141  Topic20      0.011792   ...      3.0336 -10.9461
19558  Topic20      0.011790   ...      3.0334 -10.9463
635    Topic20      0.011790   ...      3.0334 -10.9463
3678   Topic20      0.011789   ...      3.0334 -10.9463
1780   Topic20      0.011789   ...      3.0334 -10.9463
24853  Topic20      0.011789   ...      3.0334 -10.9463
21666  Topic20      0.011789   ...      3.0334 -10.9463
16602  Topic20      0.011789   ...      3.0334 -10.9463
18191  Topic20      0.011789   ...      3.0334 -10.9463
39733  Topic20      0.011789   ...      3.0334 -10.9463
11660  Topic20      0.011789   ...      3.0334 -10.9463
42829  Topic20      0.011789   ...      3.0334 -10.9463
24845  Topic20      0.011789   ...      3.0334 -10.9463
31560  Topic20      0.011789   ...      3.0334 -10.9463
31918  Topic20      0.011789   ...      3.0334 -10.9463
22938  Topic20      0.011789   ...      3.0334 -10.9463
3052   Topic20      0.011

In [8]:
##repeat the above with bigrams
dictionary_bigrams = Dictionary(df_bigrams) 
bow_corp_bigrams = [dictionary.doc2bow(file) for file in df_bigrams]
lda_model_bigrams = gensim.models.ldamodel.LdaModel(corpus=bow_corp_bigrams,
                                            id2word=dictionary_bigrams,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           random_state= 100,
                                           alpha='auto',    
                                           per_word_topics=True)
print('\nPerplexity: ', lda_model_bigrams.log_perplexity(bow_corp_bigrams))  

coherence_model_lda_bigrams = gensim.models.CoherenceModel(model=lda_model_bigrams, texts=df_bigrams, dictionary=dictionary_bigrams, coherence='u_mass')
coherence_lda_bigrams = coherence_model_lda_bigrams.get_coherence()
print('\nCoherence Score: ', coherence_lda_bigrams)


Perplexity:  -13.857436389243043

Coherence Score:  -15.401022406485135


In [9]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_bigrams, bow_corp_bigrams, dictionary_bigrams)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.393074  0.111170       1        1  44.928841
17     0.342487  0.024473       2        1  16.167297
11     0.356983  0.196479       3        1  15.724720
19     0.279258 -0.388687       4        1   7.862909
4      0.175359 -0.012445       5        1   4.860658
2     -0.011859  0.000354       6        1   2.014498
9     -0.036771  0.007944       7        1   1.819581
14    -0.061116 -0.006281       8        1   1.439191
6     -0.083764  0.003928       9        1   1.163964
10    -0.033371 -0.024667      10        1   1.031495
7     -0.117540  0.004483      11        1   0.540153
18    -0.114834  0.004310      12        1   0.515904
13    -0.113604  0.002455      13        1   0.511043
1     -0.133936  0.010336      14        1   0.386956
15    -0.132903  0.008825      15        1   0.373636
5     -0.126856  0.007250      16        1   0.248569
3     -0.137108  0.010217      17        1   0.168175
16    -0.145228  0.012292      18        1   0.159186
0     -0.148879  0.013657      19        1   0.053964
8     -0.149393  0.013909      20        1   0.029269, topic_info=      Category          Freq   ...     loglift  logprob
term                           ...                     
2331   Default   3418.000000   ...     30.0000   30.000
11823  Default   1615.000000   ...     29.0000   29.000
57     Default  14187.000000   ...     28.0000   28.000
409    Default   2268.000000   ...     27.0000   27.000
1658   Default   1959.000000   ...     26.0000   26.000
2124   Default   1921.000000   ...     25.0000   25.000
2581   Default   1909.000000   ...     24.0000   24.000
11498  Default    818.000000   ...     23.0000   23.000
990    Default   4530.000000   ...     22.0000   22.000
2101   Default   1362.000000   ...     21.0000   21.000
2916   Default   5655.000000   ...     20.0000   20.000
587    Default   5299.000000   ...     19.0000   19.000
8961   Default   1138.000000   ...     18.0000   18.000
391    Default   2806.000000   ...     17.0000   17.000
14     Default   1599.000000   ...     16.0000   16.000
227    Default   1582.000000   ...     15.0000   15.000
1102   Default   1343.000000   ...     14.0000   14.000
822    Default    844.000000   ...     13.0000   13.000
1042   Default   2053.000000   ...     12.0000   12.000
1020   Default   1197.000000   ...     11.0000   11.000
470    Default   2284.000000   ...     10.0000   10.000
1635   Default   2430.000000   ...      9.0000    9.000
4925   Default    541.000000   ...      8.0000    8.000
300    Default   1557.000000   ...      7.0000    7.000
2286   Default   1203.000000   ...      6.0000    6.000
233    Default   1576.000000   ...      5.0000    5.000
258    Default   4302.000000   ...      4.0000    4.000
130    Default   5567.000000   ...      3.0000    3.000
172    Default   2018.000000   ...      2.0000    2.000
672    Default   1545.000000   ...      1.0000    1.000
...        ...           ...   ...         ...      ...
38788  Topic20      0.003922   ...      2.8703  -11.042
35196  Topic20      0.003922   ...      2.8703  -11.042
25046  Topic20      0.003922   ...      2.8703  -11.042
39807  Topic20      0.003922   ...      2.8703  -11.042
14583  Topic20      0.003922   ...      2.8703  -11.042
33632  Topic20      0.003922   ...      2.8703  -11.042
33453  Topic20      0.003922   ...      2.8703  -11.042
25287  Topic20      0.003922   ...      2.8703  -11.042
11918  Topic20      0.003922   ...      2.8703  -11.042
31657  Topic20      0.003922   ...      2.8703  -11.042
15981  Topic20      0.003922   ...      2.8703  -11.042
8567   Topic20      0.003922   ...      2.8703  -11.042
19570  Topic20      0.003922   ...      2.8703  -11.042
30926  Topic20      0.003922   ...      2.8703  -11.042
31656  Topic20      0.003922   ...      2.8703  -11.042
31228  Topic20      0.003922   ...      2.8703  -11.042
3023   Topic20      0.003

In [10]:
##now with trigrams
dictionary_trigrams = Dictionary(df_trigrams) 
bow_corp_trigrams = [dictionary.doc2bow(file) for file in df_trigrams]
lda_model_trigrams = gensim.models.ldamodel.LdaModel(corpus=bow_corp_trigrams,
                                            id2word=dictionary_trigrams,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           random_state= 100,
                                           alpha='auto',    
                                           per_word_topics=True)
print('\nPerplexity: ', lda_model_trigrams.log_perplexity(bow_corp_trigrams))  

coherence_model_lda_trigrams = gensim.models.CoherenceModel(model=lda_model_trigrams, texts=df_trigrams, dictionary=dictionary_trigrams, coherence='u_mass')
coherence_lda_trigrams = coherence_model_lda_trigrams.get_coherence()
print('\nCoherence Score: ', coherence_lda_trigrams)


Perplexity:  -14.189050171620696

Coherence Score:  -16.200167920704946


In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_trigrams, bow_corp_trigrams, dictionary_trigrams)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.408506 -0.126642       1        1  41.536072
5     -0.375417 -0.163839       2        1  27.520994
19    -0.305386 -0.058148       3        1   9.221743
8     -0.288366  0.380174       4        1   7.859357
3     -0.005359  0.072624       5        1   2.093772
0     -0.013132  0.037248       6        1   1.931396
12     0.046091  0.003900       7        1   1.835285
7      0.025631  0.012778       8        1   1.447686
1      0.070369 -0.005281       9        1   1.243859
18     0.059370  0.004745      10        1   1.072869
15     0.087079 -0.000662      11        1   0.986903
9      0.103782 -0.014906      12        1   0.860202
16     0.078909  0.007180      13        1   0.839965
17     0.112780 -0.015434      14        1   0.672581
2      0.131183 -0.021526      15        1   0.254402
14     0.127372 -0.020742      16        1   0.225011
11     0.134258 -0.022109      17        1   0.171165
13     0.139768 -0.023122      18        1   0.125211
6      0.138106 -0.022771      19        1   0.067275
10     0.141466 -0.023467      20        1   0.034246, topic_info=      Category          Freq   ...     loglift  logprob
term                           ...                     
2916   Default   4514.000000   ...     30.0000  30.0000
2331   Default   2934.000000   ...     29.0000  29.0000
57     Default  11925.000000   ...     28.0000  28.0000
587    Default   5270.000000   ...     27.0000  27.0000
300    Default   1443.000000   ...     26.0000  26.0000
2631   Default   1320.000000   ...     25.0000  25.0000
11823  Default   1570.000000   ...     24.0000  24.0000
602    Default   2474.000000   ...     23.0000  23.0000
2066   Default   1002.000000   ...     22.0000  22.0000
2124   Default   1523.000000   ...     21.0000  21.0000
1658   Default   1602.000000   ...     20.0000  20.0000
391    Default   2774.000000   ...     19.0000  19.0000
1102   Default   1049.000000   ...     18.0000  18.0000
130    Default   4086.000000   ...     17.0000  17.0000
409    Default   1311.000000   ...     16.0000  16.0000
149    Default    854.000000   ...     15.0000  15.0000
990    Default   3957.000000   ...     14.0000  14.0000
4675   Default    752.000000   ...     13.0000  13.0000
6517   Default    734.000000   ...     12.0000  12.0000
138    Default   4243.000000   ...     11.0000  11.0000
4925   Default    736.000000   ...     10.0000  10.0000
10923  Default   2682.000000   ...      9.0000   9.0000
2581   Default   1090.000000   ...      8.0000   8.0000
284    Default   1308.000000   ...      7.0000   7.0000
6094   Default    706.000000   ...      6.0000   6.0000
580    Default   1274.000000   ...      5.0000   5.0000
2286   Default   1106.000000   ...      4.0000   4.0000
105    Default    507.000000   ...      3.0000   3.0000
2298   Default    634.000000   ...      2.0000   2.0000
1020   Default    996.000000   ...      1.0000   1.0000
...        ...           ...   ...         ...      ...
13144  Topic20      0.004053   ...      2.8030 -11.1023
7108   Topic20      0.004053   ...      2.8030 -11.1023
13738  Topic20      0.004053   ...      2.8030 -11.1023
14257  Topic20      0.004053   ...      2.8030 -11.1023
35193  Topic20      0.004053   ...      2.8030 -11.1023
37090  Topic20      0.004053   ...      2.8030 -11.1023
24112  Topic20      0.004053   ...      2.8030 -11.1023
7047   Topic20      0.004053   ...      2.8030 -11.1023
13352  Topic20      0.004053   ...      2.8030 -11.1023
36903  Topic20      0.004053   ...      2.8030 -11.1023
36904  Topic20      0.004053   ...      2.8030 -11.1023
37966  Topic20      0.004053   ...      2.8030 -11.1023
30069  Topic20      0.004053   ...      2.8030 -11.1023
22767  Topic20      0.004053   ...      2.8030 -11.1023
32722  Topic20      0.004053   ...      2.8030 -11.1023
37082  Topic20      0.004053   ...      2.8030 -11.1023
14002  Topic20      0.004

### Results of the different models

As you can see by the perplexity and coherence scores, the lda models that make use of bigrams and trigrams are slightly better than the unigram model, which one would expect (especially since the bigrams/trigrams are only implemented if they have a high enough frequency, so sparsity is not much of an issue). I still have to go through all the topics, though, so I'm not sure which one will have the best results from an analysis perspectif.

In [12]:
##ideally this would make a mallet lda model (which is supposedly slightly better...) but some error message occurs
mallet_path = r'C:\Users\temp.WINDOWS-FA0Q68D\Downloads\mallet-2.0.8\bin'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=bow_corp, num_topics=20, id2word=dictionary)
print(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = gensim.models.CoherenceModel(model=ldamallet, texts=df, dictionary=dictionary, coherence='u_mass')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)
# See trigram example

CalledProcessError: Command 'C:\Users\temp.WINDOWS-FA0Q68D\Downloads\mallet-2.0.8\bin import-file --preserve-case --keep-sequence --remove-stopwords --token-regex "\S+" --input C:\Users\TEMP~1.WIN\AppData\Local\Temp\271dd5_corpus.txt --output C:\Users\TEMP~1.WIN\AppData\Local\Temp\271dd5_corpus.mallet' returned non-zero exit status 1.

### Models for individual sources

In this section, I will first define a function that makes an LDA model prints its umass coherence and perplexity and then prints the pyldavis visualization. I then run this method on each of the sources I scraped.

In [19]:
#this function is the same as above, but I don't use bigram or trigrams
"""
@param: pickle location
@return: pyLDAvis visualization   """ 
def makeLDAmodel(picklefile):
    with open(picklefile,'rb') as f:
        df = pickle.load(f)
    dictionary = Dictionary(df) #finds allunique words i think
    bow_corp = [dictionary.doc2bow(file) for file in df] 
    lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corp,
                                            id2word=dictionary,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           random_state= 100,
                                           alpha='auto',    
                                           per_word_topics=True)
    print('\nPerplexity: ', lda_model.log_perplexity(bow_corp))  # a measure of how good the model is. lower the better.

    # Compute Coherence Score
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=df, dictionary=dictionary, coherence='u_mass')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\nCoherence Score: ', coherence_lda)
    pyLDAvis.enable_notebook()
    vis =  pyLDAvis.gensim.prepare(lda_model, bow_corp, dictionary)
    return vis

In [20]:
makeLDAmodel('bbc_lemmatized_series.pkl')

##lol at topic 6, I suck at web scraping


Perplexity:  -11.173775661178553

Coherence Score:  -5.303740610278842


C:\ProgramData\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.278718  0.091497       1        1  38.594173
5     -0.307426  0.055167       2        1  18.912333
2     -0.271498  0.056659       3        1   9.477195
3     -0.270769 -0.007226       4        1   8.755759
8     -0.226517 -0.055169       5        1   6.593843
10    -0.041804 -0.427777       6        1   5.300465
13    -0.191450  0.018170       7        1   5.297257
12    -0.046086  0.035745       8        1   3.086050
17     0.054698  0.022549       9        1   1.486724
18     0.091386  0.030083      10        1   1.253111
0      0.153544  0.023705      11        1   0.416826
7      0.160881  0.002407      12        1   0.287273
16     0.158628  0.019730      13        1   0.191544
15     0.154676  0.019186      14        1   0.184558
19     0.149400  0.018578      15        1   0.100196
9      0.146102  0.019547      16        1   0.052739
4      0.141238  0.019287      17        1   0.002507
1      0.141238  0.019287      18        1   0.002501
14     0.141238  0.019287      19        1   0.002483
11     0.141238  0.019287      20        1   0.002458, topic_info=     Category         Freq   ...     loglift  logprob
term                         ...                     
854   Default   501.000000   ...     30.0000  30.0000
405   Default  1488.000000   ...     29.0000  29.0000
1534  Default   314.000000   ...     28.0000  28.0000
797   Default  1002.000000   ...     27.0000  27.0000
4391  Default   359.000000   ...     26.0000  26.0000
1182  Default   744.000000   ...     25.0000  25.0000
111   Default   786.000000   ...     24.0000  24.0000
106   Default   497.000000   ...     23.0000  23.0000
471   Default  1094.000000   ...     22.0000  22.0000
406   Default  2716.000000   ...     21.0000  21.0000
724   Default   455.000000   ...     20.0000  20.0000
1236  Default   273.000000   ...     19.0000  19.0000
5886  Default   235.000000   ...     18.0000  18.0000
910   Default   166.000000   ...     17.0000  17.0000
843   Default   280.000000   ...     16.0000  16.0000
270   Default   243.000000   ...     15.0000  15.0000
550   Default   349.000000   ...     14.0000  14.0000
411   Default   232.000000   ...     13.0000  13.0000
289   Default   253.000000   ...     12.0000  12.0000
5059  Default   244.000000   ...     11.0000  11.0000
2549  Default   376.000000   ...     10.0000  10.0000
5924  Default   188.000000   ...      9.0000   9.0000
667   Default   299.000000   ...      8.0000   8.0000
2644  Default   110.000000   ...      7.0000   7.0000
1032  Default   345.000000   ...      6.0000   6.0000
427   Default   362.000000   ...      5.0000   5.0000
711   Default   227.000000   ...      4.0000   4.0000
1174  Default   293.000000   ...      3.0000   3.0000
1090  Default   146.000000   ...      2.0000   2.0000
293   Default   512.000000   ...      1.0000   1.0000
...       ...          ...   ...         ...      ...
406   Topic20     0.000265   ...     -5.5293  -9.7274
1     Topic20     0.000265   ...     -4.5962  -9.7274
471   Topic20     0.000265   ...     -4.6200  -9.7274
405   Topic20     0.000265   ...     -4.9277  -9.7274
162   Topic20     0.000265   ...     -4.1875  -9.7274
732   Topic20     0.000265   ...     -3.4684  -9.7274
910   Topic20     0.000265   ...     -2.7351  -9.7274
411   Topic20     0.000265   ...     -3.0717  -9.7274
795   Topic20     0.000265   ...     -0.9952  -9.7274
87    Topic20     0.000265   ...     -3.8965  -9.7274
1182  Topic20     0.000265   ...     -4.2356  -9.7274
1084  Topic20     0.000265   ...     -2.9790  -9.7274
4198  Topic20     0.000265   ...     -1.9941  -9.7274
433   Topic20     0.000265   ...     -3.7316  -9.7274
111   Topic20     0.000265   ...     -4.2906  -9.7274
2674  Topic20     0.000265   ...     -0.4147  -9.7274
248   Topic20     0.000265   ...     -4.0336  -9.7274
414   Topic20     0.000265   ...     -2.8230  -9.7274
834   Topic20  

In [21]:
makeLDAmodel('kom_lemmatized_series.pkl')
## ew topic 1, should have cleaned my data better...


Perplexity:  -11.714378145566323

Coherence Score:  -3.3017339509175363


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.268002  0.086171       1        1  23.518688
9      0.249858 -0.110396       2        1  14.245041
18     0.146362  0.046575       3        1  12.261158
2      0.209839 -0.029946       4        1  12.251605
6      0.166156  0.069192       5        1   6.338376
15     0.141538  0.019652       6        1   5.115473
3      0.151669  0.136660       7        1   4.563365
1      0.061599 -0.259479       8        1   3.971358
16    -0.023933  0.115374       9        1   2.943403
11     0.047564  0.073024      10        1   2.872663
7      0.066747  0.163842      11        1   2.473137
13     0.033488 -0.322105      12        1   2.375034
8      0.010948 -0.054414      13        1   2.059163
12    -0.204676 -0.010841      14        1   1.132418
10    -0.186835 -0.008208      15        1   1.115644
17    -0.168883  0.028805      16        1   0.903496
19    -0.206047  0.020815      17        1   0.892052
5     -0.200700 -0.000041      18        1   0.810176
14    -0.280830  0.017607      19        1   0.124622
0     -0.281865  0.017715      20        1   0.033111, topic_info=      Category          Freq   ...    loglift  logprob
term                           ...                    
0      Default  30777.000000   ...     30.000  30.0000
723    Default   5786.000000   ...     29.000  29.0000
12     Default  11303.000000   ...     28.000  28.0000
378    Default   7535.000000   ...     27.000  27.0000
1040   Default   5294.000000   ...     26.000  26.0000
1      Default  10342.000000   ...     25.000  25.0000
427    Default   3711.000000   ...     24.000  24.0000
321    Default   6024.000000   ...     23.000  23.0000
364    Default   2616.000000   ...     22.000  22.0000
343    Default   7542.000000   ...     21.000  21.0000
11     Default  13280.000000   ...     20.000  20.0000
331    Default   3032.000000   ...     19.000  19.0000
446    Default   3249.000000   ...     18.000  18.0000
264    Default   1464.000000   ...     17.000  17.0000
67     Default   2305.000000   ...     16.000  16.0000
733    Default   2488.000000   ...     15.000  15.0000
731    Default   3093.000000   ...     14.000  14.0000
2314   Default   2206.000000   ...     13.000  13.0000
1835   Default   2785.000000   ...     12.000  12.0000
329    Default   8926.000000   ...     11.000  11.0000
518    Default   4764.000000   ...     10.000  10.0000
517    Default   2377.000000   ...      9.000   9.0000
582    Default   2733.000000   ...      8.000   8.0000
701    Default   2574.000000   ...      7.000   7.0000
2      Default   2308.000000   ...      6.000   6.0000
485    Default   1871.000000   ...      5.000   5.0000
510    Default   2635.000000   ...      4.000   4.0000
648    Default   1099.000000   ...      3.000   3.0000
686    Default   1698.000000   ...      2.000   2.0000
34     Default   3829.000000   ...      1.000   1.0000
...        ...           ...   ...        ...      ...
12029  Topic20      0.008817   ...      3.123 -10.7234
5694   Topic20      0.008817   ...      3.123 -10.7234
10578  Topic20      0.008817   ...      3.123 -10.7234
14807  Topic20      0.008817   ...      3.123 -10.7234
22196  Topic20      0.008817   ...      3.123 -10.7234
40403  Topic20      0.008817   ...      3.123 -10.7234
16670  Topic20      0.008817   ...      3.123 -10.7234
38902  Topic20      0.008817   ...      3.123 -10.7234
11901  Topic20      0.008817   ...      3.123 -10.7234
11597  Topic20      0.008817   ...      3.123 -10.7234
9286   Topic20      0.008817   ...      3.123 -10.7234
23571  Topic20      0.008817   ...      3.123 -10.7234
18128  Topic20      0.008817   ...      3.123 -10.7234
21460  Topic20      0.008817   ...      3.123 -10.7234
37950  Topic20      0.008817   ...      3.123 -10.7234
3098   Topic20      0.008817   ...      3.123 -10.7234
39120  Topic20      0.008817   ...      3.123 -10.7234
10342  Topic20     

In [22]:
makeLDAmodel('tas_lemmatized_series.pkl')
##this model seems to have the best (or worst...?) UMass coherence score... 
##iirc it was the largest, so maybe larger models produce better coherence score


Perplexity:  -11.09556895854912

Coherence Score:  -6.6816518575324455


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.304842  0.033359       1        1  29.748201
1     -0.236489 -0.174230       2        1  15.450670
12    -0.280574  0.154827       3        1  13.079546
5     -0.269273 -0.029591       4        1  10.601145
16    -0.221397  0.102729       5        1   8.774821
3     -0.142478 -0.147539       6        1   4.887605
17    -0.156733 -0.253952       7        1   4.881198
19    -0.142479  0.131407       8        1   4.284723
2     -0.076125 -0.087196       9        1   2.638682
0     -0.080325  0.197890      10        1   2.271703
7      0.081548 -0.011194      11        1   1.385652
14     0.038069  0.126197      12        1   1.288879
15     0.218788 -0.006223      13        1   0.225372
9      0.222587 -0.005794      14        1   0.191842
13     0.227368 -0.005699      15        1   0.096753
11     0.226843 -0.004715      16        1   0.087572
8      0.227642 -0.004469      17        1   0.086099
10     0.222966 -0.005233      18        1   0.010491
18     0.222685 -0.005263      19        1   0.006519
6      0.222221 -0.005310      20        1   0.002524, topic_info=      Category         Freq   ...     loglift  logprob
term                          ...                     
29     Default  4653.000000   ...     30.0000  30.0000
21     Default  1730.000000   ...     29.0000  29.0000
50     Default  4864.000000   ...     28.0000  28.0000
820    Default  1133.000000   ...     27.0000  27.0000
1142   Default  1073.000000   ...     26.0000  26.0000
2119   Default  1668.000000   ...     25.0000  25.0000
2416   Default  1032.000000   ...     24.0000  24.0000
1251   Default  1601.000000   ...     23.0000  23.0000
834    Default  3007.000000   ...     22.0000  22.0000
988    Default  2478.000000   ...     21.0000  21.0000
1115   Default  1035.000000   ...     20.0000  20.0000
1096   Default   641.000000   ...     19.0000  19.0000
983    Default  2304.000000   ...     18.0000  18.0000
190    Default  1595.000000   ...     17.0000  17.0000
70     Default  2454.000000   ...     16.0000  16.0000
12     Default  2512.000000   ...     15.0000  15.0000
1302   Default   702.000000   ...     14.0000  14.0000
1037   Default   557.000000   ...     13.0000  13.0000
394    Default  2705.000000   ...     12.0000  12.0000
121    Default  2801.000000   ...     11.0000  11.0000
11     Default  1520.000000   ...     10.0000  10.0000
2291   Default   585.000000   ...      9.0000   9.0000
92     Default  1567.000000   ...      8.0000   8.0000
6472   Default   699.000000   ...      7.0000   7.0000
54     Default  3136.000000   ...      6.0000   6.0000
5652   Default   536.000000   ...      5.0000   5.0000
1036   Default   471.000000   ...      4.0000   4.0000
803    Default  1866.000000   ...      3.0000   3.0000
139    Default  1972.000000   ...      2.0000   2.0000
1312   Default  1944.000000   ...      1.0000   1.0000
...        ...          ...   ...         ...      ...
13680  Topic20     0.000514   ...      3.2842 -10.3356
10349  Topic20     0.000514   ...      3.2842 -10.3356
10261  Topic20     0.000514   ...      3.2842 -10.3356
10262  Topic20     0.000514   ...      3.2842 -10.3356
10263  Topic20     0.000514   ...      2.0227 -10.3356
10264  Topic20     0.000514   ...      3.2842 -10.3356
10265  Topic20     0.000514   ...      3.2842 -10.3356
10266  Topic20     0.000514   ...      2.9227 -10.3356
10267  Topic20     0.000514   ...      3.2842 -10.3356
10268  Topic20     0.000514   ...      3.2842 -10.3356
10269  Topic20     0.000514   ...     -1.5096 -10.3356
10270  Topic20     0.000514   ...      3.2842 -10.3356
10271  Topic20     0.000514   ...      0.1124 -10.3356
10272  Topic20     0.000514   ...      3.2842 -10.3356
10273  Topic20     0.000514   ...     -0.9103 -10.3356
10274  Topic20     0.000514   ...      3.2842 -10.3356
10275  Topic20     0.000514   ...      3.2842 -10.3356
10276  Topic20     

In [23]:
makeLDAmodel('reuters_lemmatized_series.pkl')
## all these topics seem pretty coherent so maybe higher coherence score is better
## also this seems to be the cleanest set of data, I don't see any punctuation in the topics
## although "из-за" due to probably could have been removed


Perplexity:  -9.779172000157132

Coherence Score:  -4.328431024360861


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.226334  0.065947       1        1  27.023802
8      0.222399 -0.013891       2        1  19.305685
9      0.246409  0.064118       3        1   9.997233
5      0.150078 -0.159574       4        1   7.313200
11     0.112926 -0.113556       5        1   7.274164
15     0.126505 -0.140379       6        1   5.554023
10     0.138128  0.059975       7        1   5.153017
7      0.077566  0.302167       8        1   3.885685
17    -0.046633 -0.023039       9        1   3.477117
13     0.024657 -0.041644      10        1   3.200705
16    -0.096509 -0.028651      11        1   1.735147
14    -0.040508 -0.019758      12        1   1.481353
6     -0.126499  0.000805      13        1   1.311691
0     -0.153182  0.014782      14        1   0.594631
3     -0.134091 -0.004618      15        1   0.591835
12    -0.106431 -0.017338      16        1   0.590184
4     -0.155192  0.004742      17        1   0.570026
18    -0.158048  0.018708      18        1   0.469129
19    -0.163042  0.017946      19        1   0.462972
2     -0.144869  0.013255      20        1   0.008399, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
120   Default  558.000000      россия  558.000000  30.0000  30.0000
135   Default  744.000000         сша  744.000000  29.0000  29.0000
580   Default  438.000000     санкция  438.000000  28.0000  28.0000
605   Default  476.000000       трамп  476.000000  27.0000  27.0000
252   Default  160.000000   венесуэла  160.000000  26.0000  26.0000
988   Default  145.000000   дерипаска  145.000000  25.0000  25.0000
29    Default  236.000000    компания  236.000000  24.0000  24.0000
228   Default  185.000000     украина  185.000000  23.0000  23.0000
134   Default  246.000000      страна  246.000000  22.0000  22.0000
119   Default  380.000000  российский  380.000000  21.0000  21.0000
31    Default  298.000000      москва  298.000000  20.0000  20.0000
583   Default  160.000000      сделка  160.000000  19.0000  19.0000
570   Default  133.000000       путин  133.000000  18.0000  18.0000
475   Default  204.000000   вашингтон  204.000000  17.0000  17.0000
946   Default   85.000000          en   85.000000  16.0000  16.0000
316   Default  156.000000       лидер  156.000000  15.0000  15.0000
320   Default   80.000000      мадуро   80.000000  14.0000  14.0000
1422  Default   76.000000         мэй   76.000000  13.0000  13.0000
609   Default  101.000000  украинский  101.000000  12.0000  12.0000
167   Default  184.000000      выборы  184.000000  11.0000  11.0000
1114  Default   80.000000      русала   80.000000  10.0000  10.0000
1418  Default   82.000000          ес   82.000000   9.0000   9.0000
1038  Default   98.000000      минфин   98.000000   8.0000   8.0000
121   Default  158.000000          рф  158.000000   7.0000   7.0000
79    Default  169.000000     военный  169.000000   6.0000   6.0000
2404  Default   54.000000      ракета   54.000000   5.0000   5.0000
344   Default   85.000000       нефть   85.000000   4.0000   4.0000
143   Default  104.000000     человек  104.000000   3.0000   3.0000
587   Default   87.000000  соглашение   87.000000   2.0000   2.0000
1447  Default  108.000000     встреча  108.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
988   Topic20    0.000686   дерипаска  145.317596  -2.8793  -8.7623
946   Topic20    0.000686          en   85.866524  -2.3533  -8.7624
120   Topic20    0.000686      россия  558.747192  -4.2262  -8.7624
135   Topic20    0.000686         сша  744.627319  -4.5134  -8.7624
580   Topic20    0.000686     санкция  438.671570  -3.9843  -8.7625
650   Topic20    0.000686     процент  146.292023  -2.8861  -8.7625
609   Topic20    0.000686  украинский  101.351830  -2.5191  -8.7625
119   Topic20    0.000686  российский  380.180054

In [24]:
makeLDAmodel('rad_lemmatized_series.pkl')
## some good topics, e.g. 14 is about doping (but also talks about uni):
##анти-допинговый anti-doping, спортсмен athlete, олимпийский olympic, сочи sochi, допинг any drug used for doping 


Perplexity:  -12.550488983999239

Coherence Score:  -5.575851768761869


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.297989  0.008457       1        1  23.937943
15    -0.268646 -0.107645       2        1  18.277439
5     -0.271453  0.185789       3        1  14.807510
1     -0.240559 -0.102633       4        1   9.607444
18    -0.159707 -0.279587       5        1   9.333897
9     -0.208929  0.024116       6        1   7.352009
17    -0.196667  0.062005       7        1   4.817311
10    -0.097238  0.232228       8        1   3.467858
16     0.075971  0.060957       9        1   1.091362
19     0.105215 -0.026599      10        1   1.019086
14     0.141615  0.048603      11        1   0.959120
7      0.135898 -0.082980      12        1   0.828959
6      0.135583 -0.024379      13        1   0.762839
0      0.123619  0.013461      14        1   0.761275
11     0.118917 -0.020315      15        1   0.672029
4      0.150641  0.020848      16        1   0.638870
13     0.162265 -0.010254      17        1   0.526991
8      0.196381  0.000479      18        1   0.476792
3      0.192217 -0.006844      19        1   0.385345
12     0.202866  0.004294      20        1   0.275920, topic_info=     Category         Freq   ...     loglift  logprob
term                         ...                     
152   Default  5327.000000   ...     30.0000  30.0000
467   Default  2877.000000   ...     29.0000  29.0000
433   Default  1456.000000   ...     28.0000  28.0000
151   Default  3225.000000   ...     27.0000  27.0000
2199  Default  1455.000000   ...     26.0000  26.0000
842   Default  3028.000000   ...     25.0000  25.0000
1110  Default   719.000000   ...     24.0000  24.0000
336   Default  1250.000000   ...     23.0000  23.0000
1484  Default  1017.000000   ...     22.0000  22.0000
666   Default  1097.000000   ...     21.0000  21.0000
1542  Default   843.000000   ...     20.0000  20.0000
441   Default   626.000000   ...     19.0000  19.0000
1095  Default   603.000000   ...     18.0000  18.0000
1451  Default   962.000000   ...     17.0000  17.0000
1162  Default  1000.000000   ...     16.0000  16.0000
1360  Default   862.000000   ...     15.0000  15.0000
0     Default  3525.000000   ...     14.0000  14.0000
521   Default  1145.000000   ...     13.0000  13.0000
624   Default  1296.000000   ...     12.0000  12.0000
509   Default   997.000000   ...     11.0000  11.0000
176   Default  1129.000000   ...     10.0000  10.0000
264   Default  1343.000000   ...      9.0000   9.0000
330   Default  2359.000000   ...      8.0000   8.0000
801   Default   757.000000   ...      7.0000   7.0000
374   Default  1335.000000   ...      6.0000   6.0000
644   Default   823.000000   ...      5.0000   5.0000
251   Default   831.000000   ...      4.0000   4.0000
1277  Default   787.000000   ...      3.0000   3.0000
432   Default   680.000000   ...      2.0000   2.0000
1127  Default   717.000000   ...      1.0000   1.0000
...       ...          ...   ...         ...      ...
2482  Topic20    44.911888   ...      5.8714  -3.2783
1948  Topic20    38.182049   ...      5.8677  -3.4407
1885  Topic20    33.626339   ...      5.8643  -3.5677
1324  Topic20    27.601896   ...      5.8582  -3.7651
3537  Topic20    24.892817   ...      5.8545  -3.8684
3543  Topic20    22.473444   ...      5.8505  -3.9707
2352  Topic20    19.277948   ...      5.8436  -4.1241
3495  Topic20    17.106550   ...      5.8375  -4.2436
3801  Topic20    16.629862   ...      5.8360  -4.2718
1607  Topic20    15.543180   ...      5.8321  -4.3394
3794  Topic20    15.288747   ...      5.8311  -4.3559
3806  Topic20    13.991568   ...      5.8256  -4.4446
3545  Topic20    13.623693   ...      5.8239  -4.4712
2365  Topic20    12.653815   ...      5.8188  -4.5451
3805  Topic20    12.098777   ...      5.8155  -4.5899
2479  Topic20    11.909298   ...      5.8143  -4.6057
1790  Topic20    11.626166   ...      5.8125  -4.6298
1640  Topic20    10.538971   ...      5.8046  -4.7279
1085  Topic20  